# Proceso de ETL para Dataset de Lesiones en Siniestros Viales en Buenos Aires (2019-2021)

En este archivo, nos embarcaremos en el proceso de Extracción, Transformación y Carga (ETL) de un conjunto de datos que abarca información crucial sobre lesiones ocasionadas en siniestros viales ocurridos en la ciudad de Buenos Aires, comprendiendo el periodo desde el año 2019 hasta el año 2021. Este proceso tiene como objetivo principal preparar y organizar los datos para su posterior análisis, lo que permitirá obtener insights significativos que contribuyan a mejorar la seguridad vial en esta área geográfica específica.

Los datos con los que vamos a trabajar se encuentran en la carpeta de Datasets dentro de un archivo de excel llamado lesiones

## 1. Importamos las librerias

In [69]:
import pandas as pd
import re
import numpy as np

## 2. Cargamos los datos
En esta sección, los datos serán cargados a un DataFrame de pandas y se empleará el método .head() para revisar su estructura inicial. Como los archivos se encuentran separados en dos hoajas dentro de excel, se realizara este proceso para cada una de las hojas del archivo de excel

In [70]:
lesiones_v = pd.read_excel('Datasets\lesiones.xlsx', sheet_name='VICTIMAS', na_values=['sd', 'SD'])
lesiones_h = pd.read_excel('Datasets\lesiones.xlsx', sheet_name='HECHOS', na_values=['sd', 'SD'])

In [71]:
lesiones_v.head(3)

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,Varon,57.0,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,Varon,NaN,NaN


In [72]:
lesiones_h.head(3)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9.0,NaN,14,...,-34.559658,CICLISTA,NaN,CICLISTA-SD,NaN,NaN,NaN,NaN,x,NaN
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1.0,NaN,8,...,-34.669125,AUTO,NaN,AUTO-SD,NaN,x,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2.0,NaN,8,...,-34.677556,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN


## 3. Revisamos la información de cada dataframe

En este paso, se procederá a explorar la estructura de cada uno de los dataframes previamente cargados. Esta exploración permitirá comprender la composición de los datos, identificar posibles inconsistencias o patrones relevantes, y así estar en posición de realizar las modificaciones necesarias para mejorar su calidad o adaptarlos a los requerimientos específicos del análisis o aplicación que se pretende llevar a cabo.

### Dataframe Lesiones - Victimas

In [73]:
lesiones_v.head(3)

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,Varon,57.0,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,Varon,NaN,NaN


In [74]:
lesiones_v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27605 entries, 0 to 27604
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   ID hecho          27605 non-null  object        
 1   AAA               27605 non-null  int64         
 2   MM                27605 non-null  int64         
 3   DD                27605 non-null  int64         
 4   FECHA             27605 non-null  datetime64[ns]
 5   VEHICULO_VICTIMA  11333 non-null  object        
 6   SEXO              25377 non-null  object        
 7   EDAD_VICTIMA      23134 non-null  float64       
 8   GRAVEDAD          537 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 1.9+ MB


El conjunto de datos proporcionado es un DataFrame de Pandas con 27605 entradas y 9 columnas. 

1. **ID hecho**: Esta columna parece ser un identificador único para cada entrada en el conjunto de datos. No hay valores nulos en esta columna, lo que sugiere que todas las entradas tienen un ID asociado. Sin embargo, no puedo inferir mucho más sobre esta columna sin más contexto sobre el tipo de datos que representa.

2. **AAA, MM, DD**: Estas tres columnas parecen representar la fecha del hecho en formato separado para año (AAA), mes (MM) y día (DD). No hay valores nulos en estas columnas, lo que sugiere que todas las entradas tienen una fecha asociada. Sería conveniente combinar estas tres columnas en una sola columna de fecha para facilitar el análisis temporal.

3. **FECHA**: Esta columna parece ser la fecha del hecho, representada en formato de fecha y hora. No hay valores nulos en esta columna. Sería útil verificar si los valores en esta columna coinciden con los valores en las columnas AAA, MM y DD.

4. **VEHICULO_VICTIMA**: Esta columna indica el vehículo asociado a la víctima en el incidente. Hay 11333 valores no nulos en esta columna, lo que significa que hay entradas donde el vehículo de la víctima no está registrado. Esto podría indicar que no todos los incidentes involucraron a una víctima en un vehículo o que la información sobre el vehículo de la víctima está incompleta.

5. **SEXO**: Esta columna indica el sexo de la víctima. Hay 25377 valores no nulos en esta columna, lo que sugiere que hay entradas donde no se registró el sexo de la víctima. Es posible que existan registros sin esta información o que la víctima sea de un género no binario, lo que no está representado en esta columna.

6. **EDAD_VICTIMA**: Esta columna indica la edad de la víctima. Hay 23134 valores no nulos en esta columna, lo que sugiere que hay entradas donde no se registró la edad de la víctima. Es importante tener en cuenta que esta columna está representada como un tipo de dato float64, lo que sugiere que las edades podrían tener valores decimales.

7. **GRAVEDAD**: Esta columna indica la gravedad del incidente. Hay solo 537 valores no nulos en esta columna, lo que sugiere que la mayoría de los incidentes no tienen información sobre su gravedad. Es importante tener en cuenta que la baja cantidad de valores no nulos en esta columna puede limitar su utilidad en análisis posteriores.

En resumen, este conjunto de datos parece contener información sobre incidentes, posiblemente relacionados con accidentes de tráfico u otros eventos. Hay cierta cantidad de datos faltantes en las columnas relacionadas con la víctima (como vehículo, sexo y edad), así como en la columna de gravedad del incidente.

In [75]:
# Creamos una fucnión que nos permita la normalizacion de texto dentro de un dataframe

def normalizar_texto(df, excluir_columna=None):
    # Iterar sobre todas las columnas del DataFrame
    for col in df.columns:
        # Verificar si la columna es de tipo objeto (es decir, contiene cadenas de texto)
        if df[col].dtype == 'object' and col != excluir_columna:
            # Aplicar la normalización a cada elemento de la columna
            df[col] = df[col].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', str(x).upper()).strip())
    return df

In [76]:
normalizar_texto(lesiones_v, excluir_columna='ID hecho')

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NAN,VARON,57.0,NAN
1,LC-2019-0000063,2019,1,1,2019-01-01,NAN,NAN,NaN,NAN
2,LC-2019-0000079,2019,1,1,2019-01-01,NAN,VARON,NaN,NAN
3,LC-2019-0000082,2019,1,1,2019-01-01,NAN,VARON,45.0,NAN
4,LC-2019-0000082,2019,1,1,2019-01-01,NAN,MUJER,45.0,NAN
...,...,...,...,...,...,...,...,...,...
27600,LC-2021-0451911,2021,9,11,2021-09-11,TRANSPORTE PUBLICO,VARON,87.0,NAN
27601,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,MUJER,60.0,NAN
27602,LC-2021-0530228,2021,10,25,2021-10-25,TRANSPORTE PUBLICO,MUJER,32.0,NAN
27603,LC-2021-0201378,2021,5,2,2021-05-02,MOTO,VARON,32.0,NAN


In [77]:
# Como ya contamos con una columna de fecha, procedemos a eliminar las columnas que sobran dentro del dataframe
lesiones_v = lesiones_v.drop(columns=['AAA', 'MM', 'DD'])
lesiones_v.head(3)

,ID hecho,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019-01-01,NAN,VARON,57.0,NAN
1,LC-2019-0000063,2019-01-01,NAN,NAN,NaN,NAN
2,LC-2019-0000079,2019-01-01,NAN,VARON,NaN,NAN


In [78]:
lesiones_v['VEHICULO_VICTIMA'].value_counts()

VEHICULO_VICTIMA
NAN                   16272
MOTO                   4778
CICLISTA               1759
PEATON                 1685
AUTO                   1641
TRANSPORTE PUBLICO      680
TAXI                    241
UTILITARIO              231
MOVIL                   131
MIXTO                   113
CAMION                   40
MONOPATIN                27
OTRO                      7
Name: count, dtype: int64

In [79]:
lesiones_v['SEXO'].value_counts()

SEXO
VARON    17022
MUJER     8355
NAN       2228
Name: count, dtype: int64

In [80]:
lesiones_v['SEXO'] = lesiones_v['SEXO'].replace({'VARON': 'MASCULINO', 'MUJER': 'FEMENINO'})
lesiones_v['SEXO'].value_counts()

SEXO
MASCULINO    17022
FEMENINO      8355
NAN           2228
Name: count, dtype: int64

In [81]:
lesiones_v['EDAD_VICTIMA'].value_counts()

EDAD_VICTIMA
28.0     812
29.0     805
27.0     776
30.0     765
26.0     761
        ... 
0.0        2
95.0       2
96.0       1
97.0       1
100.0      1
Name: count, Length: 99, dtype: int64

In [82]:
lesiones_v['GRAVEDAD'].value_counts()

GRAVEDAD
NAN      27068
GRAVE      537
Name: count, dtype: int64

In [83]:
lesiones_v.rename(columns={'ID hecho': 'ID_hecho', 'EDAD_VICTIMA': 'EDAD'}, inplace=True)

In [84]:
# Guardamos nuestro dataframe modificado
lesiones_v.to_csv('CSV\lesiones_victimas.csv', index=False)

### Dataframe Lesiones - Hechos

In [85]:
lesiones_h.head(3)

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9.0,NaN,14,...,-34.559658,CICLISTA,NaN,CICLISTA-SD,NaN,NaN,NaN,NaN,x,NaN
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1.0,NaN,8,...,-34.669125,AUTO,NaN,AUTO-SD,NaN,x,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2.0,NaN,8,...,-34.677556,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
lesiones_h.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23785 entries, 0 to 23784
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     23785 non-null  object 
 1   n_victimas             23785 non-null  int64  
 2   aaaa                   23785 non-null  int64  
 3   mm                     23785 non-null  int64  
 4   dd                     23785 non-null  int64  
 5   fecha                  23785 non-null  object 
 6   hora                   23776 non-null  object 
 7   franja_hora            23776 non-null  float64
 8   direccion_normalizada  12917 non-null  object 
 9   comuna                 22770 non-null  object 
 10  tipo_calle             12740 non-null  object 
 11  otra_direccion         5490 non-null   object 
 12  calle                  12867 non-null  object 
 13  altura                 12771 non-null  float64
 14  cruce                  9407 non-null   object 
 15  ge

Este análisis se basa en un conjunto de datos representado por un DataFrame de Pandas con 23,785 entradas y 27 columnas. A continuación, se detalla el análisis de cada columna:

1. **id**: Identificador único para cada registro. Todos los registros tienen valores no nulos, lo que sugiere que no hay duplicados en esta columna.
  
2. **n_victimas**: Número de víctimas involucradas en el incidente. Todos los registros tienen valores no nulos y son de tipo entero. Se espera que los valores sean mayores o iguales a cero.

3. **aaaa, mm, dd**: Columnas separadas que representan la fecha del incidente, con el año (aaaa), mes (mm) y día (dd). Todos los registros tienen valores no nulos y son de tipo entero. Se puede combinar esta información en una sola columna de fecha.

4. **fecha**: Columna que parece ser una concatenación de las columnas aaaa, mm y dd en formato de fecha. Todos los registros tienen valores no nulos y son de tipo objeto (cadena de caracteres).

5. **hora**: Hora del incidente. Algunos registros tienen valores nulos. Debería convertirse a tipo de datos de hora.

6. **franja_hora**: Franja horaria del incidente. Algunos registros tienen valores nulos. Es de tipo float64, lo que sugiere que podría representar la hora en un formato numérico.

7. **direccion_normalizada**: Dirección normalizada del incidente. Hay registros con valores nulos. Es de tipo objeto (cadena de caracteres).

8. **comuna**: Comuna del incidente. Algunos registros tienen valores nulos. Es de tipo objeto (cadena de caracteres).

9. **tipo_calle**: Tipo de calle. Algunos registros tienen valores nulos. Es de tipo objeto (cadena de caracteres).

10. **otra_direccion**: Otra dirección relacionada con el incidente. Hay registros con valores nulos. Es de tipo objeto (cadena de caracteres).

11. **calle**: Nombre de la calle. Algunos registros tienen valores nulos. Es de tipo objeto (cadena de caracteres).

12. **altura**: Altura de la calle. Algunos registros tienen valores nulos. Es de tipo float64.

13. **cruce**: Nombre de la calle de cruce. Hay registros con valores nulos. Es de tipo objeto (cadena de caracteres).

14. **geocodificacion_CABA**: Geocodificación de la Ciudad Autónoma de Buenos Aires. Algunos registros tienen valores nulos. Es de tipo objeto (cadena de caracteres).

15. **longitud**: Longitud geográfica. Algunos registros tienen valores nulos. Es de tipo float64.

16. **latitud**: Latitud geográfica. Algunos registros tienen valores nulos. Es de tipo float64.

17. **victima**: Información sobre la víctima. Hay registros con valores nulos. Es de tipo objeto (cadena de caracteres).

18. **acusado**: Información sobre el acusado. Hay registros con valores nulos. Es de tipo objeto (cadena de caracteres).

19. **participantes**: Lista de participantes involucrados en el incidente. Todos los registros tienen valores no nulos. Es de tipo objeto (cadena de caracteres).

20. **moto, auto, transporte_publico, camion, ciclista**: Indicadores de la presencia de vehículos en el incidente. Hay registros con valores nulos. Son de tipo objeto (cadena de caracteres).

21. **gravedad**: Nivel de gravedad del incidente. La mayoría de los registros tienen valores nulos. Es de tipo objeto (cadena de caracteres).

En general, se observa una cantidad significativa de valores nulos en varias columnas, lo que indica que se necesitará algún tipo de limpieza de datos antes de realizar un análisis más detallado. Además, se pueden realizar varias transformaciones de datos, como la combinación de columnas de fecha y hora, y la conversión de tipos de datos adecuados para análisis posteriores.

In [87]:
# Utilizando máscaras booleanas para filtrar las columnas a eliminar
columnas_a_eliminar = ['aaaa','mm','dd', 'hora', 'otra_direccion',
                       'calle','altura','cruce','geocodificacion_CABA',
                       'direccion_normalizada','moto','auto',
                       'transporte_publico','camion','ciclista']

# Eliminar las columnas innecesarias utilizando máscaras booleanas
lesiones_h = lesiones_h.loc[:, ~lesiones_h.columns.isin(columnas_a_eliminar)]

In [88]:
lesiones_h.head(3)

,id,n_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latutid,victima,acusado,participantes,gravedad
0,LC-2019-0000179,1,2019-01-01 00:00:00,9.0,14,NaN,-58.408911,-34.559658,CICLISTA,NaN,CICLISTA-SD,NaN
1,LC-2019-0000053,1,2019-01-01 00:00:00,1.0,8,NaN,-58.443510,-34.669125,AUTO,NaN,AUTO-SD,NaN
2,LC-2019-0000063,1,2019-01-01 00:00:00,2.0,8,NaN,-58.468335,-34.677556,NaN,NaN,SD-SD,NaN


In [89]:
# Crear nuevas columnas 'vehiculo1' y 'vehiculo2' a partir de 'participantes'
lesiones_h[['AFECTADO_1', 'AFECTADO_2']] = lesiones_h['participantes'].str.split('-', expand=True)

# Reemplazar 'sd' con NaN en ambas columnas
lesiones_h[['AFECTADO_1', 'AFECTADO_2']] = lesiones_h[['AFECTADO_1', 'AFECTADO_2']].replace('SD',np.nan)

#Borrar la columna participantes
lesiones_h.drop('participantes',axis=1, inplace=True)

In [90]:
lesiones_h.head(3)

,id,n_victimas,fecha,franja_hora,comuna,tipo_calle,longitud,latutid,victima,acusado,gravedad,AFECTADO_1,AFECTADO_2
0,LC-2019-0000179,1,2019-01-01 00:00:00,9.0,14,NaN,-58.408911,-34.559658,CICLISTA,NaN,NaN,CICLISTA,NaN
1,LC-2019-0000053,1,2019-01-01 00:00:00,1.0,8,NaN,-58.443510,-34.669125,AUTO,NaN,NaN,AUTO,NaN
2,LC-2019-0000063,1,2019-01-01 00:00:00,2.0,8,NaN,-58.468335,-34.677556,NaN,NaN,NaN,NaN,NaN


In [91]:
lesiones_h.columns = map(str.upper, lesiones_h.columns)

In [92]:
lesiones_h.head(3)

,ID,N_VICTIMAS,FECHA,FRANJA_HORA,COMUNA,TIPO_CALLE,LONGITUD,LATUTID,VICTIMA,ACUSADO,GRAVEDAD,AFECTADO_1,AFECTADO_2
0,LC-2019-0000179,1,2019-01-01 00:00:00,9.0,14,NaN,-58.408911,-34.559658,CICLISTA,NaN,NaN,CICLISTA,NaN
1,LC-2019-0000053,1,2019-01-01 00:00:00,1.0,8,NaN,-58.443510,-34.669125,AUTO,NaN,NaN,AUTO,NaN
2,LC-2019-0000063,1,2019-01-01 00:00:00,2.0,8,NaN,-58.468335,-34.677556,NaN,NaN,NaN,NaN,NaN


In [93]:
lesiones_h['FECHA'] = pd.to_datetime(lesiones_h['FECHA']).dt.date

In [94]:
# Cambiamos los nombres de las columnas
lesiones_h.rename(columns={'LATUTID': 'LATITUD', 'TIPO_CALLE': 'TIPO_DE_CALLE'}, inplace=True)
lesiones_h.head(2)

,ID,N_VICTIMAS,FECHA,FRANJA_HORA,COMUNA,TIPO_DE_CALLE,LONGITUD,LATITUD,VICTIMA,ACUSADO,GRAVEDAD,AFECTADO_1,AFECTADO_2
0,LC-2019-0000179,1,2019-01-01,9.0,14,NaN,-58.408911,-34.559658,CICLISTA,NaN,NaN,CICLISTA,NaN
1,LC-2019-0000053,1,2019-01-01,1.0,8,NaN,-58.443510,-34.669125,AUTO,NaN,NaN,AUTO,NaN


In [95]:
lesiones_h['FRANJA_HORA'].value_counts()

FRANJA_HORA
17.0    1734
16.0    1712
14.0    1627
13.0    1610
15.0    1558
18.0    1554
12.0    1550
19.0    1350
11.0    1270
20.0    1140
10.0    1131
9.0     1117
8.0     1036
21.0     974
0.0      807
22.0     762
7.0      684
23.0     574
6.0      423
1.0      341
5.0      271
2.0      233
4.0      163
3.0      155
Name: count, dtype: int64

In [96]:
lesiones_h['COMUNA'].value_counts()

COMUNA
1                  2526
15                 1908
4                  1729
3                  1718
9                  1674
14                 1581
7                  1516
12                 1440
11                 1411
13                 1355
10                 1347
5                  1226
8                  1062
6                   957
2                   878
No Especificada     442
Name: count, dtype: int64

In [97]:
lesiones_h['TIPO_DE_CALLE'].value_counts()

TIPO_DE_CALLE
AVENIDA      6775
CALLE        5383
GRAL PAZ      538
AUTOPISTA      44
Name: count, dtype: int64

In [98]:
lesiones_h['VICTIMA'].value_counts()

VICTIMA
MOTO                  5461
CICLISTA              2209
AUTO                  2033
PEATON                2026
TRANSPORTE PUBLICO     678
CAMIONETA              189
TAXI                   182
MOVIL                  100
CAMION                  79
MIXTO                   42
MONOPATIN               28
OTRO                    13
UTILITARIO              12
Name: count, dtype: int64

In [99]:
lesiones_h['ACUSADO'].value_counts()

ACUSADO
AUTO                  4641
TRANSPORTE PUBLICO     975
CAMIONETA              872
MOTO                   625
TAXI                   591
CAMION                 288
OBJETO FIJO            279
MOVIL                   81
CICLISTA                77
UTILITARIO              28
OTRO                    24
PEATON                  13
MONOPATIN                3
Name: count, dtype: int64

In [100]:
lesiones_h['AFECTADO_1'].value_counts()

AFECTADO_1
MOTO                  5461
CICLISTA              2209
AUTO                  2033
PEATON                2026
TRANSPORTE PUBLICO     678
CAMIONETA              189
TAXI                   182
MOVIL                  100
CAMION                  79
MIXTO                   42
MONOPATIN               28
OTRO                    13
UTILITARIO              12
Name: count, dtype: int64

In [101]:
lesiones_h['AFECTADO_2'].value_counts()

AFECTADO_2
AUTO                  4641
TRANSPORTE PUBLICO     975
CAMIONETA              872
MOTO                   625
TAXI                   591
CAMION                 288
OBJETO FIJO            279
MOVIL                   81
CICLISTA                77
UTILITARIO              28
OTRO                    24
PEATON                  13
MONOPATIN                3
Name: count, dtype: int64

In [102]:
# Guardamos nuestro Dataframe modificado
lesiones_h.to_csv('CSV\lesiones_hechos.csv', index=False)